# LSTM GAN Training

In [1]:
cd ../..

/Users/hao/workspace/hpi_de/4th_Semester/Applied Machine Learning/sensor-data-gans


## imports

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from numpy.random import randn
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, LSTM, LeakyReLU, Reshape
from tensorflow.keras.optimizers import Adam
from gans.gan_trainer import GanTrainer

## Create Discriminator and Generator Networks
methods which creates and returns generator and discriminator

In [ ]:
def create_discriminator(input_shape, optimizer=Adam(learning_rate=0.0001, clipnorm=1.0, decay=1e-8)):
    discriminator = Sequential()
    discriminator.add(LSTM(5, activation='relu', return_sequences=True, input_shape=input_shape))
    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return discriminator

In [ ]:
def create_generator(latent_dim, units=3, small=True):
    if small:
        generator = Sequential()
        generator.add(Dense(window_size, kernel_initializer='he_uniform', input_shape=(latent_dim, )))
        generator.add(LeakyReLU())
        generator.add(Reshape((window_size, 1)))
        generator.add(LSTM(units, return_sequences=True, kernel_initializer='he_uniform'))
        generator.add(LeakyReLU())
        generator.add(LSTM(3, return_sequences=True, kernel_initializer='he_uniform', activation='linear'))
    else:
        generator = Sequential()
        generator.add(Dense(window_size, kernel_initializer='he_uniform', input_shape=(latent_dim, )))
        generator.add(LeakyReLU())
        generator.add(Reshape((window_size, 1)))
        generator.add(LSTM(12, return_sequences=True, kernel_initializer='he_uniform'))
        generator.add(LeakyReLU())
        generator.add(LSTM(9, return_sequences=True, kernel_initializer='he_uniform'))
        generator.add(LeakyReLU())
        generator.add(LSTM(6, return_sequences=True, kernel_initializer='he_uniform'))
        generator.add(LeakyReLU())
        generator.add(LSTM(3, return_sequences=True, kernel_initializer='he_uniform', activation='linear'))
    
    return generator

## Training Routine

### parameters for data loading and preprocessing

In [ ]:
train_path = './datasets/mydata/train_df.h5'
val_path = './datasets/mydata/val_df.h5'
test_path = './datasets/mydata/test_df.h5'

act_id = 0

window_size = 5*50
step_size = int(window_size/2)
method ='sliding'
col_names = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z', 'userAcceleration.c']
input_cols_train=['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
input_cols_eval=['userAcceleration.c']

In [ ]:
steps = 1000
batch_size = 32
eval_step = 100
random = False
label_smoothing = False

## Training routines with different hyper parameters

In [ ]:
for small in [True, False]:
    for act_id in range(0,3):
        for latent_dim in [32, 64, 128, 256]:
            for unit in [2, 3, 4, 5, 6, 7]:
                out_dir = './out/lstm/act_id-{}/{}_ld-{}_u-{}_small-{}'.format(act_id, datetime.now(), latent_dim, unit, small)
                
                print("New: ", out_dir, 'Act_id:', act_id)

                gan_trainer = GanTrainer(out_dir=out_dir)
                gan_trainer.init_data(train_path=train_path,
                              val_path=val_path,
                              test_path=test_path,
                              act_id=act_id,
                              window_size=window_size,
                              step_size=step_size,
                              method=method,
                              col_names=col_names,
                              input_cols_train=input_cols_train,
                              input_cols_eval=input_cols_eval
                             )
                input_shape = gan_trainer.x_train[0].shape

                discriminator = create_discriminator(input_shape)
                generator = create_generator(latent_dim=latent_dim, small=small)
                gan_trainer.create_gan(generator=generator, discriminator=discriminator)

                gan_trainer.train_gan(steps=steps, 
                                              batch_size=batch_size,
                                              eval_step=eval_step,
                                              random=random, 
                                              label_smoothing=label_smoothing)